In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from logitplots import plt_confusion_matrix, plt_decision_boundaries, plt_correlation_matrix

In [2]:
df= pd.read_csv("chipVariantCalling_run1.tsv", sep='\t')
df = df.drop_duplicates()
df = df.dropna()
df = df.drop(df[df.chipOrControl == 'Unknown'].index)

## Create Dummy's for Categorical Variables

In [3]:
categories = df['SYMBOL'].unique()
lencat = len(categories) 
for a in categories: 

    df[a] = pd.Series(df['SYMBOL']==a).astype(int)

categories = df['sampleTimePt'].unique()
lencat = len(categories) 
for a in categories: 

    df[a] = pd.Series(df['sampleTimePt']==a).astype(int)
    
categories = df['gender'].unique()
lencat = len(categories) 
for a in categories: 

    df[a] = pd.Series(df['gender']==a).astype(int)
    
categories = df['IMPACT'].unique()
lencat = len(categories) 
for a in categories: 

    df[a] = pd.Series(df['IMPACT']==a).astype(int)

df = df.drop(['SYMBOL', 'sampleTimePt','gender','IMPACT'], axis = 1)

## Remove Identifiers from Data Frame we Will use to Make Models

In [4]:
final_df = df.drop(['d.barcode','MSID'], axis = 1)
final_df = final_df.reset_index(drop = True)
final_df.head(6)

,DP,VD,AF,HIAF,loci,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,...,IDH2,SRSF2,RP11-661A12.9,TLK2,Baseline,Y3,Male,Female,MODERATE,HIGH
0,7281,26,0.0036,0.0033,chr1:1747196_T/C,2:2,3644:3596,12:14,34.0,1.182210,...,0,0,0,0,1,0,1,0,1,0
1,7282,29,0.0040,0.0036,chr1:1747250_T/C,2:2,3639:3606,15:14,32.7,1.061729,...,0,0,0,0,1,0,1,0,1,0
2,7282,24,0.0033,0.0033,chr1:1747256_T/C,2:2,3626:3607,12:12,34.9,1.005270,...,0,0,0,0,1,0,1,0,1,0
3,178,2,0.0112,0.0120,chr1:115256571_T/C,2:2,88:88,1:1,37.0,1.000000,...,0,0,0,0,1,0,1,0,1,0
4,1773,6,0.0034,0.0035,chr1:115258674_T/C,2:2,889:874,3:3,37.0,1.017150,...,0,0,0,0,1,0,1,0,1,0
5,1773,7,0.0039,0.0029,chr1:115258679_T/C,2:2,891:873,3:4,29.6,1.360580,...,0,0,0,0,1,0,1,0,1,0


## Convert ratios to Decimals

In [5]:
the_list = []                             # create an empty list
for i in range(len(final_df['BIAS'])):      # loop over all values
    ratio = final_df['BIAS'][i]     # read the ratio
    first = ''
    second = ''
    index = ratio.index(':')              # find the index of ':' in 'ratio'
    for a in range(0, index):             # get the value of the first part of the ratio
        first += ratio[a]
    for b in range(index+1, len(ratio)):  # get the value of the second part of the ratio
        second += ratio[b]
    total = int(first) + int(second)      # total = first + second
    if total == 0:                        # if the total = 0, then set decimal to be 10.0
        the_list += [10.0]
    else:
        decimal = int(first) / total      # else, set decimal to be first/total
        the_list += [decimal]


final_df['bias_decimal'] = the_list

In [6]:
the_list = []                             # create an empty list
for i in range(len(final_df['REFBIAS'])):      # loop over all values
    ratio = final_df['REFBIAS'][i]     # read the ratio
    first = ''
    second = ''
    index = ratio.index(':')              # find the index of ':' in 'ratio'
    for a in range(0, index):             # get the value of the first part of the ratio
        first += ratio[a]
    for b in range(index+1, len(ratio)):  # get the value of the second part of the ratio
        second += ratio[b]
    total = int(first) + int(second)      # total = first + second
    if total == 0:                        # if the total = 0, then set decimal to be 10.0
        the_list += [10.0]
    else:
        decimal = int(first) / total      # else, set decimal to be first/total
        the_list += [decimal]


final_df['REFBIAS'] = the_list

In [7]:
the_list = []                             # create an empty list
for i in range(len(final_df['VARBIAS'])):      # loop over all values
    ratio = final_df['VARBIAS'][i]     # read the ratio
    first = ''
    second = ''
    index = ratio.index(':')              # find the index of ':' in 'ratio'
    for a in range(0, index):             # get the value of the first part of the ratio
        first += ratio[a]
    for b in range(index+1, len(ratio)):  # get the value of the second part of the ratio
        second += ratio[b]
    total = int(first) + int(second)      # total = first + second
    if total == 0:                        # if the total = 0, then set decimal to be 10.0
        the_list += [10.0]
    else:
        decimal = int(first) / total      # else, set decimal to be first/total
        the_list += [decimal]


final_df['VARBIAS'] = the_list

## Normalisation

In [8]:
mean = final_df['DP'].mean()
SD = final_df['DP'].mean()
the_list = []  
for i in final_df['DP']:
    (i - mean)/SD
    

## Random Forest

In [9]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression 

In [10]:
final_df.head()

,DP,VD,AF,HIAF,loci,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,...,SRSF2,RP11-661A12.9,TLK2,Baseline,Y3,Male,Female,MODERATE,HIGH,bias_decimal
0,7281,26,0.0036,0.0033,chr1:1747196_T/C,2:2,0.503315,0.461538,34.0,1.182210,...,0,0,0,1,0,1,0,1,0,0.5
1,7282,29,0.0040,0.0036,chr1:1747250_T/C,2:2,0.502277,0.517241,32.7,1.061729,...,0,0,0,1,0,1,0,1,0,0.5
2,7282,24,0.0033,0.0033,chr1:1747256_T/C,2:2,0.501313,0.500000,34.9,1.005270,...,0,0,0,1,0,1,0,1,0,0.5
3,178,2,0.0112,0.0120,chr1:115256571_T/C,2:2,0.500000,0.500000,37.0,1.000000,...,0,0,0,1,0,1,0,1,0,0.5
4,1773,6,0.0034,0.0035,chr1:115258674_T/C,2:2,0.504254,0.500000,37.0,1.017150,...,0,0,0,1,0,1,0,1,0,0.5


In [11]:
X = final_df.drop(['chipOrControl', 'loci','BIAS'], axis = 1) 
y = final_df['chipOrControl']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=0)
rf = RandomForestClassifier(n_estimators=200, max_samples=.5).fit(X_train, y_train)
Y_pred = rf.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, Y_pred),3)))

Accuracy 0.731


In [12]:
for name, score in zip(X.columns, rf.feature_importances_):
    print(name,np.round(score,3))

DP 0.184
VD 0.058
AF 0.105
HIAF 0.107
REFBIAS 0.148
VARBIAS 0.029
QUAL 0.054
ODDRATIO 0.141
GNB1 0.001
NRAS 0.004
NOTCH2 0.004
DNMT3A 0.014
ASXL2 0.002
CERKL 0.001
SF3B1 0.006
TET2 0.016
SNX18 0.001
BRAF 0.004
RAD21 0.006
JAK2 0.002
TET1 0.001
CBL 0.004
KRAS 0.003
PTPN11 0.003
CREBBP 0.006
TP53 0.008
DNAH2 0.002
PPM1D 0.007
ASXL1 0.013
CHMP4B 0.003
GNAS 0.003
LIPI 0.001
U2AF1 0.003
ZFX 0.0
BCOR 0.005
BCORL1 0.004
BRCC3 0.005
SRY 0.001
ZFY 0.0
AMELY 0.0
LY9 0.0
SYNE1 0.0
AC078842.3 0.0
NOTCH1 0.001
IDH2 0.001
SRSF2 0.0
RP11-661A12.9 0.0
TLK2 0.0
Baseline 0.006
Y3 0.006
Male 0.007
Female 0.007
MODERATE 0.005
HIGH 0.005
bias_decimal 0.003
